In [1]:
# a notebook to check that metaworld is working, and a testbed for new RL algos 
import metaworld

SEED = 0  # some seed number here
benchmark = metaworld.ML1('pick-place-v2', seed=SEED)

In [2]:
import random

print(metaworld.ML1.ENV_NAMES)  # Check out the available environments

ml1 = metaworld.ML1('pick-place-v2') # Construct the benchmark, sampling tasks

env = ml1.train_classes['pick-place-v2']()  # Create an environment with task `pick_place`
task = random.choice(ml1.train_tasks)
env.set_task(task)  # Set task

obs, info = env.reset()  # Reset environment
a = env.action_space.sample()  # Sample an action
obs, reward, terminal, truncated, info = env.step(a)  # Step the environment with the sampled random action


['assembly-v2', 'basketball-v2', 'bin-picking-v2', 'box-close-v2', 'button-press-topdown-v2', 'button-press-topdown-wall-v2', 'button-press-v2', 'button-press-wall-v2', 'coffee-button-v2', 'coffee-pull-v2', 'coffee-push-v2', 'dial-turn-v2', 'disassemble-v2', 'door-close-v2', 'door-lock-v2', 'door-open-v2', 'door-unlock-v2', 'hand-insert-v2', 'drawer-close-v2', 'drawer-open-v2', 'faucet-open-v2', 'faucet-close-v2', 'hammer-v2', 'handle-press-side-v2', 'handle-press-v2', 'handle-pull-side-v2', 'handle-pull-v2', 'lever-pull-v2', 'peg-insert-side-v2', 'pick-place-wall-v2', 'pick-out-of-hole-v2', 'reach-v2', 'push-back-v2', 'push-v2', 'pick-place-v2', 'plate-slide-v2', 'plate-slide-side-v2', 'plate-slide-back-v2', 'plate-slide-back-side-v2', 'peg-unplug-side-v2', 'soccer-v2', 'stick-push-v2', 'stick-pull-v2', 'push-wall-v2', 'reach-wall-v2', 'shelf-place-v2', 'sweep-into-v2', 'sweep-v2', 'window-open-v2', 'window-close-v2']


In [3]:
ml10 = metaworld.ML10() # Construct the benchmark, sampling tasks

training_envs = []
for name, env_cls in ml10.train_classes.items():
  env = env_cls()
  task = random.choice([task for task in ml10.train_tasks
                        if task.env_name == name])
  env.set_task(task)
  training_envs.append(env)

for env in training_envs:
  obs, info = env.reset()  # Reset environment
  a = env.action_space.sample()  # Sample an action
  obs, reward, terminated, truncated, info = env.step(a)  # Step the environment with the sampled random action

In [18]:
# now lets try continual_world
# run single

from continualworld.envs import get_single_env, get_cl_env
from continualworld.utils.utils import get_activation_from_str
from continualworld.sac.models import MlpActor
from continualworld.utils.run_utils import get_sac_class

steps_per_task = 1000000
activation="lrelu"
num_tasks=1
hidden_sizes=[256, 256, 256, 256]
tasks=['basketball-v2']
# logger = EpochLogger(args["logger_output"], config=args, group_id=args["group_id"])

logger = None
steps = steps_per_task * len(tasks)
num_heads = num_tasks
train_env = get_cl_env(tasks, steps_per_task)
test_envs = [
    get_single_env(task, one_hot_idx=i, one_hot_len=num_tasks) for i, task in enumerate(tasks)
]

actor_kwargs = dict(
    hidden_sizes=hidden_sizes,
    activation=get_activation_from_str(activation),
    use_layer_norm=True,
    num_heads=num_heads,
    hide_task_id=True,
)
critic_kwargs = dict(
    hidden_sizes=hidden_sizes,
    activation=get_activation_from_str(activation),
    use_layer_norm=True,
    num_heads=num_heads,
    hide_task_id=True,
)

actor_cl = MlpActor

In [19]:
from continualworld.utils.enums import BufferType
lr=0.0025
alpha='auto'
buffer_type="fifo"
gamma=0.99
seed=13
clipnorm=None
target_output_std = 0.089
agent_policy_exploration = False
cl_reg_coef=0.0  # strength of MAS/EWC
regularize_critic = False

vanilla_sac_kwargs = {
    "env": train_env,
    "test_envs": test_envs,
    "logger": logger,
    "seed": seed,
    "steps": steps,
    "log_every": 100,
    "replay_size": 10000,
    "batch_size": 10,
    "actor_cl": actor_cl,
    "actor_kwargs": actor_kwargs,
    "critic_kwargs": critic_kwargs,
    "buffer_type": BufferType(buffer_type),
    "reset_buffer_on_task_change": True,
    "reset_optimizer_on_task_change": True,
    "lr": lr,
    "alpha": alpha,
    "reset_critic_on_task_change": True,
    "clipnorm": clipnorm,
    "gamma": gamma,
    "target_output_std": target_output_std,
    "agent_policy_exploration": agent_policy_exploration,
}


In [20]:
cl_method = None  # 'mas'

sac_class = get_sac_class(cl_method)
if cl_method is None:
    sac = sac_class(**vanilla_sac_kwargs)
elif cl_method in ["l2", "ewc", "mas"]:
    sac = sac_class(
        **vanilla_sac_kwargs, 
        cl_reg_coef=cl_reg_coef, 
        regularize_critic=regularize_critic
    )



In [22]:
sac.run()


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.